<a href="https://colab.research.google.com/github/ynusinovich/streamflow-prediction/blob/main/ARE/HYSETS_DARTS_training_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Streamflow Data/'

Mounted at /content/drive


In [2]:
# !pip install pyyaml==5.4.1
!pip install xarray==0.18.2
!pip install torch==1.8.1  # there appears to be a version conflict with the current version of pytorch and cuDNN
!pip install darts==0.21.0
# !pip install -U matplotlib==3.1.3
# !pip install cudnn==8.3.2

import xarray as xr
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 807 kB 2.1 MB/s 
  Attempting uninstall: xarray
    Found existing installation: xarray 0.20.2
    Uninstalling xarray-0.20.2:
      Successfully uninstalled xarray-0.20.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires xarray>=0.20, but you have xarray 0.18.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that 

In [5]:
#@title Load Pickled Data

import gzip, pickle

lcrop = True

pickle_name = "hysets_preprocessed_timseries{}.gz".format('_crop' if lcrop else '')

with gzip.open(data_root+pickle_name, 'rb') as gz:
    pickle_dict = pickle.load(gz)

ts_series_train = pickle_dict['ts_series_train']
cov_series_train = pickle_dict[ 'cov_series_train']
ts_series_test = pickle_dict['ts_series_test']
cov_series_test = pickle_dict['cov_series_test']


In [ ]:
#@title Train a LSTM Model with the Data


In [ ]:
## load existing LSTM model
from darts.models import RNNModel

VERSION = 2
N = 10
acc_type = 'GPU'

filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.{acc_type}.pt"

model = RNNModel.load(filepath)
print("Epochs trained:", model.epochs_trained)

Epochs trained: 0


In [6]:
# ## Create a new LSTM model

# from darts.models import RNNModel

# VERSION = 2
# acc_type = 'GPU'

# pl_trainer_kwargs = dict(accelerator=acc_type.lower())
# if acc_type == 'GPU':
#   pl_trainer_kwargs["gpus"] = [0]

# # simplified version of the Kratzert model
# model = RNNModel(
#     model="LSTM",
#     input_chunk_length=270,
#     training_length=280,
#     hidden_dim=128, # 256
#     n_rnn_layers=1,
#     dropout=0.1, # 0.4
#     n_epochs=1,
#     pl_trainer_kwargs=pl_trainer_kwargs,
# )

# # train model for one epoch
# model.fit(ts_series_train, future_covariates=cov_series_train, verbose=True)

# N = 0
# filepath = data_root + f"/HYSETS_LSTM_v{VERSION}.{N}.{acc_type}.pt"
# model.save(filepath)
# print(f"\n   ***   Saved trained model version {VERSION}.{N} to Google Drive (trained with {acc_type})   ***   \n",f"'{filepath}'")
# print("Epochs trained:", model.epochs_trained)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.c

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.



   ***   Saved trained model version 2.0 to Google Drive (trained with GPU)   ***   
 '/content/drive/My Drive/Streamflow Data//HYSETS_LSTM_v2.0.GPU.pt'
Epochs trained: 1


In [ ]:
# train model for some epochs

epochs = 5
save_interval = 1 # int(epochs/10)

for i in range(epochs):
    N += 1
    print(f"\n   ***   Training Iteration {N}   ***")
    model.fit(ts_series_train, future_covariates=cov_series_train, verbose=True)
    if N % save_interval == 0:
        filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.{acc_type}.pt"
        model.save(filepath)
        print(f"\n   ***   Saved trained model version {VERSION}.{N} to Google Drive (trained with {acc_type})   ***   \n",f"'{filepath}'")


   ***   Training Iteration 1   ***


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.



   ***   Saved trained model version 2.1 to Google Drive (trained with GPU)   ***   
 '/content/drive/My Drive/Streamflow Data//HYSETS_LSTM_v2.1.GPU.pt'

   ***   Training Iteration 2   ***


...

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.



   ***   Saved trained model version 2.4 to Google Drive (trained with GPU)   ***   
 '/content/drive/My Drive/Streamflow Data//HYSETS_LSTM_v2.4.GPU.pt'

   ***   Training Iteration 5   ***


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training: 0it [00:00, ?it/s]

In [ ]:
%%time
#@title Test Model Prediction

# sid = 0  # long
# sid = 2  # longer
# sid = 3  # short
fcst_len = 100
input_len = model.model_params['input_chunk_length']

n_ws = len(ts_series_test)
pred_list = []
val_list = []

for sid in range(n_ws):

    ts_test = ts_series_test[sid]
    cov_test = cov_series_test[sid]

    # generate prediction series    
    series_len = len(ts_test)
    if input_len+fcst_len > series_len:
        print(f"   ---   Skipping Timeseries {sid+1} (of {n_ws}): too short for forecast")
    else:
        print(f"   ***   Validating Timeseries {sid+1} (of {n_ws}); length: {series_len} ...")

        init_idxs = np.arange(input_len, series_len-fcst_len+1, dtype=np.int16)

        ts_list = [ts_test[ii-input_len:ii] for ii in init_idxs]
        val_ts = [ts_test[ii:ii+fcst_len] for ii in init_idxs]
        val_list.append(val_ts)  # used in next cell, alongside pred

        cov_list = [cov_test[ii-input_len:ii+fcst_len] for ii in init_idxs]

        # run model prediction
        #print(" running model prediction ...", end='')
        pred = model.predict(n=fcst_len, series=ts_list, future_covariates=cov_list, verbose=None,)
        pred_list.append(pred)
        #print(" done.")



In [14]:
pred[1]

<TimeSeries (DataArray) (time: 100, component: 1, sample: 1)>
array([[[-0.69340765]],

       [[-0.70326686]],

       [[-0.7127067 ]],

       [[-0.722563  ]],

       [[-0.7306572 ]],

       [[-0.73632205]],

       [[-0.73881257]],

       [[-0.7462859 ]],

       [[-0.7514994 ]],

       [[-0.75800204]],

...

       [[-1.1315714 ]],

       [[-1.1139473 ]],

       [[-1.0889761 ]],

       [[-1.0684336 ]],

       [[-1.0638288 ]],

       [[-1.055729  ]],

       [[-1.0503744 ]],

       [[-1.0516324 ]],

       [[-1.0530975 ]],

       [[-1.0495784 ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-03-05 2013-03-06 ... 2013-06-12
  * component  (component) object '0'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  Centroid_Lat_deg_N  Drainage_Area_...
    hierarchy:          None

In [ ]:
%%time
## compute SMAPE

assert len(pred_list) == len(val_list)
n_pred = len(pred_list)
smape = np.zeros((n_pred, fcst_len))
nse = np.zeros((n_pred, fcst_len))
n_fcst_array = np.zeros((n_pred,1))

# loop over timeseries
for sid in range(n_pred):
# for sid in range(6,8):

    # ts_xdv = ts_series_test[sid].data_array()
    # print("Test Series Time Period:", ts_xdv.time[0].values,ts_xdv.time[-1].values)
    pred = pred_list[sid]
    val_ts = val_list[sid]
    
    n_fcst = len(pred)
    n_fcst_array[sid,0] = n_fcst
    print(f"Validating Wateshed {sid+1} (of {n_ws}); Number of Forecasts: {n_fcst}")

    smape_array = np.zeros((n_fcst, fcst_len))
    nse_up = np.zeros((n_fcst, fcst_len))
    nse_down = np.zeros((n_fcst, fcst_len))

    for i in range(n_fcst):
        val_xdv = val_ts[i].data_array()
        fcst_xdv = pred[i].data_array()
        if i in (0, n_fcst-1):
            assert val_xdv.time[0].values == fcst_xdv.time[0].values, (val_xdv.time[0].values,fcst_xdv.time[0].values)
            assert val_xdv.time[-1].values == fcst_xdv.time[-1].values, (val_xdv.time[-1].values,fcst_xdv.time[-1].values)
            # print("  Forecast #{i}:", val_xdv.time[0].values,val_xdv.time[-1].values)    
            # print("  Forecast #{i}:", fcst_xdv.time[0].values,fcst_xdv.time[-1].values)
        # renormalize
        ts_mean = fcst_xdv.attrs['static_covariates']['discharge_mean'][0]
        ts_std = fcst_xdv.attrs['static_covariates']['discharge_std'][0]
        val_data = val_xdv.values.squeeze()*ts_std + ts_mean
        fcst_data = fcst_xdv.values.squeeze()*ts_std + ts_mean
        fcst_data = fcst_data.clip(0,None)
        # fcst_data[:] = val_data[0]  # test for persistence forecast
        denominator = (np.abs(val_data) + np.abs(fcst_data))
        den_0 = ( denominator == 0 ).sum()
        if den_0 > 0:
            print(f"Zero-streamflow encountered {den_0}x")
            denominator[denominator == 0] = 1  # number doesn't matter, since enumerator is also zero
        smape_array[i,:] = 200 * np.abs(val_data - fcst_data) / denominator        
        nse_up[i,:] = np.square(val_data - fcst_data)
        nse_down[i,:] = np.square(val_data - ts_mean)  # already zero-centered
        
    smape[sid,:] = smape_array.mean(axis=0)
    nse[sid,:] = 1 - ( nse_up.sum(axis=0) / nse_down.sum(axis=0) )

n_fcst_sum = n_fcst_array.sum()
smape_avg = np.sum(smape * n_fcst_array, axis=0) / n_fcst_sum
smape_med = np.median(smape, axis=0)
nse_avg = np.sum(nse * n_fcst_array, axis=0) / n_fcst_sum
nse_med = np.median(nse, axis=0)

results = {" Median SMAPE: ": smape_med, "Average SMAPE: ": smape_avg, "   Median NSE: ": nse_med,"  Average NSE: ": nse_avg,}
for name,values in results.items():
    print('\n'+name, end='')
    for n in values: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')

In [9]:
#@title Previous Un-trained Forecast from Version 2 (longer training length)
for name,values in results.items():
    print('\n'+name, end='')
    for n in values: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')


 Median SMAPE:    24%,   36%,   52%,   61%,   67%,   72%,   79%,   84%,   86%,   91%,   97%,  102%,  107%,  111%,  115%,  119%,  122%,  124%,  126%,  127%,  129%,  132%,  134%,  135%,  135%,  134%,  133%,  133%,  132%,  132%,  133%,  134%,  134%,  134%,  132%,  133%,  134%,  135%,  135%,  136%,  136%,  137%,  138%,  138%,  138%,  138%,  139%,  141%,  141%,  141%,  145%,  147%,  146%,  146%,  145%,  145%,  147%,  148%,  149%,  149%,  150%,  152%,  152%,  152%,  153%,  153%,  153%,  154%,  154%,  155%,  155%,  155%,  156%,  156%,  157%,  157%,  157%,  158%,  158%,  158%,  159%,  159%,  159%,  160%,  160%,  160%,  161%,  161%,  161%,  161%,  161%,  162%,  162%,  163%,  163%,  164%,  164%,  164%,  165%,  165%,
Average SMAPE:    28%,   42%,   51%,   59%,   65%,   70%,   74%,   78%,   82%,   85%,   88%,   90%,   93%,   95%,   97%,   99%,  101%,  103%,  104%,  106%,  107%,  109%,  110%,  111%,  112%,  113%,  114%,  115%,  116%,  117%,  118%,  119%,  120%,  120%,  121%,  122%,  123%,  123%,  

In [ ]:
#@title Previous Trained Forecast from Version 1
for name,values in results.items():
    print('\n'+name, end='')
    for n in values: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')


 Median SMAPE:    19%,   29%,   37%,   44%,   51%,   57%,   65%,   71%,   76%,   80%,   83%,   86%,   87%,   89%,   90%,   92%,   95%,   99%,   99%,   99%,  100%,  101%,  102%,  104%,  105%,  106%,  107%,  108%,  109%,  109%,  110%,  110%,  111%,  111%,  112%,  112%,  112%,  112%,  112%,  113%,  113%,  113%,  113%,  114%,  115%,  116%,  118%,  119%,  120%,  121%,  122%,  122%,  123%,  123%,  123%,  124%,  124%,  124%,  124%,  124%,  124%,  124%,  124%,  125%,  125%,  125%,  126%,  127%,  128%,  129%,  130%,  131%,  131%,  131%,  131%,  131%,  131%,  131%,  131%,  131%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  132%,  131%,  131%,  131%,  131%,  131%,  132%,  132%,
Average SMAPE:    24%,   35%,   43%,   50%,   56%,   61%,   65%,   69%,   73%,   76%,   78%,   81%,   83%,   85%,   87%,   89%,   90%,   92%,   93%,   95%,   96%,   97%,   98%,  100%,  101%,  102%,  102%,  103%,  104%,  105%,  106%,  106%,  107%,  108%,  108%,  109%,  110%,  110%,  

In [ ]:
#@title Skill of Persistence Forecast
for name,values in results.items():
    print('\n'+name, end='')
    for n in values[1:]: print(f' {n:4.0f}%,' if 'SMAPE' in name else f' {n:5.2f},', end='')


 Median SMAPE:     9%,   15%,   20%,   24%,   28%,   31%,   34%,   37%,   40%,   43%,   45%,   48%,   49%,   51%,   53%,   54%,   56%,   58%,   60%,   62%,   63%,   64%,   65%,   67%,   67%,   69%,   70%,   71%,   72%,   74%,   74%,   75%,   76%,   77%,   78%,   80%,   81%,   81%,   82%,   83%,   83%,   83%,   84%,   85%,   86%,   87%,   88%,   88%,   89%,   90%,   91%,   91%,   92%,   93%,   94%,   94%,   95%,   96%,   96%,   95%,   94%,   94%,   94%,   94%,  100%,  100%,  101%,  101%,  101%,  102%,  102%,  102%,  103%,  103%,  104%,  104%,  105%,  106%,  107%,  107%,  108%,  108%,  109%,  109%,  109%,  109%,  110%,  110%,  110%,  110%,  111%,  111%,  112%,  112%,  112%,  111%,  111%,  111%,  111%,
Average SMAPE:     9%,   15%,   20%,   25%,   28%,   32%,   35%,   37%,   40%,   42%,   45%,   47%,   49%,   51%,   52%,   54%,   56%,   57%,   59%,   60%,   62%,   63%,   65%,   66%,   67%,   68%,   70%,   71%,   72%,   73%,   74%,   75%,   76%,   77%,   78%,   79%,   80%,   81%,   82%,  